In [1]:
import pandas as pd
import numpy as np

In [11]:
# import urllib library
from urllib.request import urlopen
  
# import json
import json
# store the URL in url as 
# parameter for urlopen
url = "https://classes.cornell.edu/api/2.0/search/classes.json?roster=SP22&subject=CS" # get SP 22 CS classes
  
# store the response of URL
response = urlopen(url)
  
# storing the JSON response 
# from url in data
data_json = json.loads(response.read())


In [12]:
classes = data_json['data']['classes']

In [13]:
url2 = 'https://classes.cornell.edu/api/2.0/config/subjects.json?roster=SP22'
response2 = urlopen(url2)
subjects_json = json.loads(response2.read())

In [18]:
subject_list = []
for i in subjects_json['data']['subjects']:
    subject_list.append(i['value'])


In [19]:
# CS 1110, Bracy, start time, end time, location, description, prereqs/coreqs, semester offered
df = []
missedClasses = 0

for subject in subject_list: #, "ECON"]"ENGL", "PHIL"]:

    url = f"https://classes.cornell.edu/api/2.0/search/classes.json?roster=SP22&subject={subject}"
    response = urlopen(url)

    # storing the JSON response 
    # from url in data
    data_json = json.loads(response.read())

    classes = data_json['data']['classes']

    for class_ in classes:
        #print(class_)
        try:
            #if (int(class_['catalogNbr'])) > 5900:
            #    continue
            #print(class_)
            obj = class_['enrollGroups'][0]['classSections'][0]['meetings'][0]

            obj2 = obj['instructors'][0]
            #print(obj)

            startTime = obj['timeStart']
            endTime = obj['timeEnd']
            location = obj['facilityDescr']
            x = [class_['subject'] , class_['catalogNbr'] , class_['titleLong'], obj2['firstName'].strip() + ' ' + obj2['lastName'].strip(), startTime, endTime, location, class_['description'], class_['catalogPrereqCoreq'], class_['catalogWhenOffered']]
            df.append(x)
        except:
            missedClasses += 1
print(missedClasses)

211


In [20]:
df1 = pd.DataFrame(np.array(df))
df1.shape

(4327, 10)

In [22]:
df1 = df1.rename(columns={0:"Dept", 1:"Number", 2:"Course_Name", 3:"Professor", 4:"Start_Time", 5:"End_Time", 6:"Location",7:"Description",8:"Pre/coreqs",9:"Offered"})

In [23]:
median_grades = pd.read_csv("Cleaned_Medians.csv")

In [24]:
median_grades

,Unnamed: 0,Dept + Number,Professor,Median Grade,Semester (Ex: SP21),# of Students,Notes,Semester
0,1,AAS 2130,Chang,A,FA21,52,"crosslisted with AMST 2640, HIST 2640",0.0
1,289,ENGL 1158 (FWS: Asian American Food Writing),Rhee,A,FA21,17,NaN,0.0
2,585,PLBIO 2300,Nixon,A,FA21,13,NaN,0.0
3,287,ENGL 1105,Various (FWS),A+,FA21,49,NaN,0.0
4,286,EDUC 2610,Steinhall,A,FA21,86,NaN,0.0
...,...,...,...,...,...,...,...,...
519,227,DEA 3510,Hedge,B,FA18,14,NaN,6.0
520,175,COMM 2200,Humphreys,A-,FA18,95,Cookie Cutter Essays,6.0
521,140,CEE 4510,Richardson,A-,FA17,44,NaN,8.0
522,171,COML 1109,Various (FWS),A-,FA17,NaN,NaN,8.0


In [25]:
median_grades = median_grades[['Dept + Number', 'Median Grade', 'Semester (Ex: SP21)', 'Semester']]

In [26]:
df1["Dept + Number"] = df1["Dept"] + ' ' + df1["Number"]

In [27]:
df1['Dept + Number'].value_counts()

CEE 6590      1
VIEN 1105     1
CHEME 7410    1
HD 1120       1
VETMI 7441    1
             ..
EDUC 4826     1
EAS 4790      1
GOVT 7937     1
ROMS 6525     1
COMM 6450     1
Name: Dept + Number, Length: 4327, dtype: int64

In [28]:
df2 = df1.merge(median_grades, on='Dept + Number', how="outer")

In [29]:
df2 = df2.dropna(subset=['Dept'])

In [30]:
df2

,Dept,Number,Course_Name,Professor,Start_Time,End_Time,Location,Description,Pre/coreqs,Offered,Dept + Number,Median Grade,Semester (Ex: SP21),Semester
0,AAS,1100,Introduction to Asian American Studies,Derek Chang,09:40AM,10:55AM,Rockefeller Hall 132,This interdisciplinary course offers an introd...,,Spring.,AAS 1100,NaN,NaN,NaN
1,AAS,2620,Introduction to Asian American Literature,Sunn Wong,11:25AM,12:40PM,Goldwin Smith Hall 142,This course will introduce both a variety of w...,,Spring.,AAS 2620,NaN,NaN,NaN
2,AAS,3030,Asians in the Americas: A Comparative Perspective,Viranjini Munasinghe,02:45PM,04:00PM,McGraw Hall B65,The common perception of ethnicity is that it ...,,Spring.,AAS 3030,NaN,NaN,NaN
3,AAS,4020,U.S. Cultures of War and Empire,Christine Balance,11:20AM,01:50PM,Sibley Hall B12,This course examines the history and afterlive...,,Spring.,AAS 4020,NaN,NaN,NaN
4,AAS,4550,Race and the University,Derek Chang,01:30PM,04:30PM,Goldwin Smith Hall 122,"What is a university, what does it do, and how...",,Spring.,AAS 4550,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4322,YORUB,2111,Intermediate Yoruba II,Adeolu Ademoyo,09:05AM,09:55AM,Stimson Hall G24B,Intermediate Yoruba II is a follow-up to Inter...,Prerequisite: YORUB 1108 and YORUB 1109.,Spring.,YORUB 2111,NaN,NaN,NaN
4323,YORUB,3111,Advanced Yoruba II,Adeolu Ademoyo,02:45PM,04:00PM,None,This course will help students expand their un...,Prerequisite: YORUB 3110.,Spring.,YORUB 3111,NaN,NaN,NaN
4324,ZULU,1116,Elementary Zulu II,Angelika Kraemer,11:35AM,12:25PM,To Be Assigned,Development of communication skills through di...,,Spring.,ZULU 1116,NaN,NaN,NaN
4325,ZULU,2117,Intermediate Zulu II,Angelika Kraemer,09:25AM,10:15AM,To Be Assigned,Students read longer texts from popular media ...,,Spring.,ZULU 2117,NaN,NaN,NaN


In [31]:
df2.to_csv("Classes_With_Medians.csv")